In [53]:
import pandas_datareader.data as web
import pandas as pd
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import backtesting

BILDF = web.DataReader('BIL', 'stooq')
BILDF = web.DataReader('BND', 'stooq')
QQQDF = web.DataReader('QQQ', 'stooq')
UPRODF = web.DataReader('UPRO', 'stooq')
EIFDF = web.DataReader('EIF', 'stooq')



In [54]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [56]:
class RiskOnRiskOff(Strategy):
    n=60 # 60 day lookback period

    def init(self):
        # compute the cumulative 60 day returns of bnd and bil
        self.bnd_cumret = self.I(lambda x: x['Close'] / x['Close'].iloc[0])(BND)
        self.bil_cumret = self.I(lambda x: x['Close'] / x['Close'].iloc[0])(BIL)
        

    def next(self):
        # if bnd has outperformed bil over the last 60 days, buy UPRO and QQQ equally else BUY IEF
        if self.bnd_cumret[-1] > self.bil_cumret[-1]:
            self.position.close()
            self.buy('UPRO', 0.5)
            self.buy('QQQ', 0.5)
        else:
            self.position.close()
            self.buy('IEF', 1.0)

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

bt = Backtest(BILDF, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats
bt.plot()


/var/folders/yc/lrz1jj2j5vgfg6xn03_69x480000gn/T/ipykernel_15162/2607825325.py:44: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(BILDF, SmaCross, cash=10_000, commission=.002)
/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/homebrew/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values 

TypeError: bokeh.models.tools.Toolbar() got multiple values for keyword argument 'logo'